In [28]:
import sys
sys.path.append("../")  # path contains python_file.py

In [18]:
import iris

import numpy as np
import scipy.stats

In [19]:
%load_ext autoreload
%autoreload 2
import PACKAGE_NAME.math_helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
era5 = iris.load("testing_data/ERA5.nc")
obs = era5[0]

cm_hist = iris.load_cube(
    "testing_data/CMIP6/medium_area/historical_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_19790116-20141216_v20200729.nc"
)
cm_future = iris.load_cube(
    "testing_data/CMIP6/medium_area/SSP3-7_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_ssp370_r1i1p1f1_gr_20150116-20501216_v20201218.nc"
)

/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(
/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(


In [21]:
# Regrid
cm_hist = cm_hist.regrid(obs, iris.analysis.Linear())
cm_future = cm_future.regrid(obs, iris.analysis.Linear())

In [22]:
# Get data
obs = np.array(obs.data[:, 0, :, :])
cm_hist = cm_hist.data
cm_future = cm_future.data

Test ISIMIP:

In [23]:
obs = obs[:, 1, 1]
cm_hist = cm_hist[:, 1, 1]
cm_future = cm_future[:, 1, 1]

In [24]:
from PACKAGE_NAME.isimip import ISIMIP

temp_debiaser = ISIMIP(variable="tas")
#temp_debiaser.trend_preservation = "mixed"
"""temp_debiaser.variable = "prsnratio"
temp_debiaser.trend_preservation = "mixed"
temp_debiaser.lower_bound = 0
temp_debiaser.upper_bound = 300
temp_debiaser.lower_threshold = 50
temp_debiaser.upper_treshold = 290
temp_debiaser.powerlaw_exponent_step4 = 2"""

'temp_debiaser.variable = "prsnratio"\ntemp_debiaser.trend_preservation = "mixed"\ntemp_debiaser.lower_bound = 0\ntemp_debiaser.upper_bound = 300\ntemp_debiaser.lower_threshold = 50\ntemp_debiaser.upper_treshold = 290\ntemp_debiaser.powerlaw_exponent_step4 = 2'

In [25]:
obs = obs[obs > 0]  # Two obs-outliers. Need to check for that.

In [26]:
obs, cm_hist, cm_future = temp_debiaser.step2(obs, cm_hist, cm_future)
obs, cm_hist, cm_future = temp_debiaser.step4(obs, cm_hist, cm_future)
obs_future = temp_debiaser.step5(obs, cm_hist, cm_future)
test = temp_debiaser.step6(obs, obs_future, cm_hist, cm_future)

TypeError: only integer scalar arrays can be converted to a scalar index

In [27]:
x = temp_debiaser.apply_location(obs, cm_hist, cm_future)

TypeError: only integer scalar arrays can be converted to a scalar index

In [112]:
x

array([278.92354859, 278.03362182, 280.71583275, 280.80814741,
       285.71244905, 291.451553  , 292.51733714, 293.3272245 ,
       288.127578  , 284.52473855, 282.88308501, 278.92219873,
       278.5360191 , 278.75153198, 280.15205676, 282.05041075,
       284.31815308, 292.69812499, 294.68632804, 294.10531959,
       291.08766708, 286.05645532, 281.289237  , 280.08444938,
       274.71348775, 279.55485356, 281.9911085 , 281.14265463,
       283.66548857, 289.92344403, 295.03612203, 295.8150738 ,
       289.06416776, 283.76405773, 281.10146752, 279.28634443,
       278.91669666, 279.45187223, 282.10493788, 280.30717767,
       285.08207801, 291.33237727, 290.29355897, 293.89455545,
       290.47948576, 283.32623153, 281.55159515, 280.10067015,
       279.89294248, 279.07046807, 281.35507973, 282.80187836,
       286.24736515, 294.28545888, 293.21054003, 291.73249417,
       292.1921984 , 284.88800188, 282.21884531, 279.9719695 ,
       278.64416468, 280.65899574, 281.53241595, 284.80

Missing from ISIMIP: 

- Steps 1 and 8 (scaling for rsds)
- Steps 2 and 7 (trend removal)
- Code-consistency, user-options, non-parametric, interface, etc.